In [1]:
#https://github.com/OceanParcels/GlobalMassBudget/blob/main/create_river_input.py

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import timeit
import matplotlib.colors as colors
import csv
import shapefile
import cartopy
import glob
from datetime import datetime
import os

In [2]:
import geopandas as gpd
import pandas as pd

In [3]:
mask_land = xr.open_dataset('../../data/output_data/masks/mask_land_NEMO0083.nc')['mask_land']
mask_land_nan = mask_land.copy().values.astype(float)
mask_land_nan[~mask_land] = np.nan
mask_lons,mask_lats = mask_land['lon'].values, mask_land['lat'].values
#fieldMesh_x,fieldMesh_y = np.meshgrid(lons.ravel(),lats.ravel())

In [4]:
files_fisheries = sorted(glob.glob('../../data/release/GlobalFishingWatch/fleet*/*') )

In [5]:
# date_array = pd.date_range(datetime(2020,1,1),datetime(2021,1,1),freq='MS')    
# delta_date_array = (( np.array(date_array) - np.datetime64(datetime(1970,1,1)) ) ).astype('timedelta64[s]').astype(int)


In [ ]:
files_fisheries[0][-12:-3]

"\n## Fisheries data is provided on a 0.01x0.01 grid with extent [-180,180]x[-90,90]\n# e.g. # print(np.unique(sorted(data_fisheries['cell_ll_lat'].values)))\n# and  # print(np.unique(sorted(data_fisheries['cell_ll_lon'].values)))\n# We will create a grid to bin these on\ndlon = 0.01\ndlat = 0.01\nlons = np.arange(-180,180,step=dlon)\nlats = np.arange(-90,90,step=dlat)\nfieldMesh_x,fieldMesh_y = np.meshgrid(lons,lats)\n\n\nlons_edge = lons - .5*dlon\nlats_edge = lats - .5*dlat\nlons_edge = np.append(lons_edge,lons_edge[-1]+dlon)\nlats_edge = np.append(lats_edge,lats_edge[-1]+dlat)\n#mesh_plot_x,mesh_plot_y = np.meshgrid(lons_edge,lats_edge)\n\n\ndlon_f,dlat_f = 0.01,0.01\n\n## I think this is an inefficient way of doing it\n# input_fisheries = np.zeros([len(date_array)-1,fieldMesh_x.shape[0],fieldMesh_y.shape[1]])\n# for file_ in files_fisheries:\n    \n#     date = datetime.fromisoformat(os.path.basename(file_).split('.')[0])\n#     if date.day == 1:\n#         print(date)\n#     delt

In [7]:
# Instead, let's store the lat/lon, date, and summed hours
data_fisheries = []
for file_ in files_fisheries:
    data_fisheries_day = pd.read_csv(file_)
    data_fisheries_day = data_fisheries_day[data_fisheries_day['fishing_hours'] > 0]
    data_fisheries.append(data_fisheries_day)

data_fisheries = pd.concat(data_fisheries, axis=0, ignore_index=True)

In [9]:
data_fisheries

,date,cell_ll_lat,cell_ll_lon,flag,geartype,hours,fishing_hours,mmsi_present
0,2020-01-01,30.00,-75.50,USA,fishing,0.0702,0.0702,1
1,2020-01-01,30.00,-75.41,USA,fishing,0.0636,0.0636,1
2,2020-01-01,-34.00,17.57,ZAF,fishing,0.1847,0.1847,1
3,2020-01-01,-34.00,17.58,ZAF,fishing,0.1677,0.1677,1
4,2020-01-01,-36.00,20.29,ZAF,fishing,0.2672,0.2336,1
...,...,...,...,...,...,...,...,...
91732748,2020-12-31,-33.17,17.08,ZAF,fishing,0.1683,0.1683,1
91732749,2020-12-31,-33.17,17.14,ZAF,fishing,0.2980,0.2980,2
91732750,2020-12-31,-33.17,17.15,ZAF,fishing,0.2327,0.2327,2
91732751,2020-12-31,-33.17,17.01,ZAF,fishing,0.2222,0.2222,1


In [24]:
agg_data_fisheries = data_fisheries.groupby(['cell_ll_lat', 'cell_ll_lon', 'date'])['fishing_hours'].agg('sum').reset_index()

In [25]:
agg_data_fisheries['date'] = pd.to_datetime(agg_data_fisheries['date'])

In [26]:
agg_data_fisheries['month'] = agg_data_fisheries['date'].values.astype('datetime64[M]')


In [28]:

agg_data_fisheries = agg_data_fisheries.groupby(['cell_ll_lat', 'cell_ll_lon', 'month'])['fishing_hours'].agg('sum').reset_index()

In [30]:
agg_data_fisheries.to_csv('../../data/release/GlobalFishingWatch/fleet_monthly_aggregated.csv')

## This chops it down from 70gb for one year to 2gb!

In [ ]:
# def to_netcdf(output_filename,data,data_name,lons,lats,time,explanation='',encode=False):
#     '''
#     All data is written to netcdf files to speed up computations
#     '''
#     dict_data = {}
#     for name_, data_ in zip(data_name, data):
#         dict_data[name_] = (( "time", "lat", "lon"), data_ )
#     dict_data['explanation'] = explanation
        
#     ds = xr.Dataset(
#         dict_data,
#         coords={
#             "lon": lons,
#             "lat": lats,
#             "time": time,
#         },
#     )   
#     if encode:
#         ds.to_netcdf(output_filename,encoding={name_: {'dtype': 'int16', 'scale_factor': 0.1, '_FillValue': -9999}})
#     else:
#         ds.to_netcdf(output_filename)

# to_netcdf('../../data/release/generated_files/input_fisheries.nc',[input_fisheries],['fishing_hours'],lons,lats,date_array[:-1],explanation='Total fishing hours per month, calculated from the Global Fishing Watch v2 dataset')

In [ ]:
plt.pcolormesh(mask_lons,mask_lats,mask_land_nan)
plt.show()

plt.pcolormesh(fieldMesh_x,fieldMesh_y,np.sum(input_fisheries, axis=0),norm=colors.LogNorm(vmin=1e-3,vmax=1e2))
plt.colorbar()
plt.show()
